# Types of model

## Description

All `simple` models are instances of the `simple.Model` class.
The simplest way to create them is by calling `simple.Model(parameters, log_likelihood)`, as demonstrated in [Getting started](./getting-started.ipynb).

For many use cases, the mean of the likelihood is generated by a call to a physical forward model.
`simple` provides the `simple.ForwardModel` class to handle this.
Forward models have the same functionality as regular models, but they also accept a `forward` argument at initialization.
The main advantage of forward models is that their `forward` attribute will handle both dictionary and array parameters, and that they can easily generate prior predictive samples through the `get_prior_pred()` method.

Another common scenario is a forward model where the likelihood is Gaussian and independent.
To save users from writing the log of a Gaussian everytime they define a new model, `simple` has a `GaussianForwardModel` class.
This is really just a `ForwardModel` with a Gaussian log-likelihood.
The likelihood has a signature `GaussianForwardModel.log_likelihood(parameters, data, err, *args, **kwargs)`.
The output of the forward model is compared against `data`, and the error is `err`. If `parameters` contains a `sigma` element, it is added in quadrature to `err`. All other arguments are passed to the forward model.

## Usage

Below, we demonstrate the same linear model implemented with each type of `simple` model.


In [1]:
import numpy as np


def forward(parameters, x):
    m, b = parameters["m"], parameters["b"]
    return m * x + b


def log_likelihood(parameters, x, y, yerr):
    mu = forward(parameters, x)
    sigma = np.sqrt(parameters["sigma"] ** 2 + yerr**2)
    return -0.5 * np.sum(((y - mu) / sigma) ** 2 + np.log(2 * np.pi * sigma**2))

In [2]:
from scipy.stats import loguniform, uniform

from simple import ForwardModel, GaussianForwardModel, Model
from simple.distributions import ScipyDistribution

parameters = {
    "m": ScipyDistribution(uniform(-10, 20)),
    "b": ScipyDistribution(uniform(-10, 20)),
    "sigma": ScipyDistribution(loguniform(1e-5, 100)),
}

model = Model(parameters, log_likelihood)
forward_model = ForwardModel(parameters, log_likelihood, forward)
Gaussian_model = GaussianForwardModel(parameters, forward)

And we can check that the models work as expected.

In [3]:
rng = np.random.default_rng()
x = np.sort(10 * rng.random(100))
m_true = 1.338
b_true = -0.45
truths = {"m": m_true, "b": b_true, "sigma": None}
y_true = m_true * x + b_true
yerr = 0.1 + 0.5 * rng.random(x.size)

y = y_true + 2 * yerr * rng.normal(size=x.size)
test_point = {"m": 1.0, "b": 0, "sigma": 1.0}
print("Model Log likelihood", model.log_likelihood(test_point, x, y, yerr))
print("Forward Log likelihood", forward_model.log_likelihood(test_point, x, y, yerr))
print(
    "GaussianForward Log likelihood",
    Gaussian_model.log_likelihood(test_point, y, yerr, x),
)

Model Log likelihood -205.62993788686032
Forward Log likelihood -205.62993788686032
GaussianForward Log likelihood -205.62993788686032
